In [ ]:
!pip install tsplib95

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.3 MB/s eta 0:00:00
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0:
      Successfully uninstalled tabulate-0.9.0
  Attempting uninstall: networkx
    Found existing installation: networkx 3.3
    Uninstalling networkx-3.3:
      Successfully uninstalled networkx-3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.21.0 requires tabulate>=0.9, but you have tabulate 0.8.10 which is incompatible.


In [ ]:
# Run 1: population_size=100, max_gen=6, elite_count=2, mutation_prob=0.01
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=100, max_gen=6, elite_count=2, mutation_prob=0.01):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/rat99.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=100, max_gen=6, elite_count=2, mutation_prob=0.01)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 7434.726885502602
Generation 2: Best Cost = 7434.726885502602
Generation 3: Best Cost = 7434.726885502602
Generation 4: Best Cost = 7434.726885502602
Generation 5: Best Cost = 7434.726885502602
Generation 6: Best Cost = 7434.726885502602
Best solution found: [57 54 48 67 22 35 45 53 64 72 76 77 20 16 61 12 24 68 84 27 21  2 52  3
  6 17 71  0 26 58 13 49 60 40 46 78 18 65  8 69  7 30 44 88 33  5 41 98
 96  9  4 28 51 97 42 11 82 73 29 19 70 62 90 66 74 91 94 81 56 59 31 79
 55 63 83 34 23 38 36 87 47 50 43 15 32 75 80 86 39 92 95 10  1 25 37 14
 93 89 85]
Cost of the best solution: 74347.26885502602
Average time per generation: 0.11778791745503744


In [ ]:
# Run 2: population_size=100, max_gen=6, elite_count=3, mutation_prob=0.02
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=100, max_gen=6, elite_count=3, mutation_prob=0.02):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/rat99.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=100, max_gen=6, elite_count=3, mutation_prob=0.02)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 7297.519790192927
Generation 2: Best Cost = 7297.519790192927
Generation 3: Best Cost = 7297.519790192927
Generation 4: Best Cost = 7297.519790192927
Generation 5: Best Cost = 7297.519790192927
Generation 6: Best Cost = 7297.519790192927
Best solution found: [40 74  7 50 23  8 17 67 76 90 61 16 75 10  3  6 38 86 63 87 12  5 48 47
 94 46 55 11 72 35 25 83 65 96 34 28 97 62 45 15 30 19 73 81 71 42 84 13
 69 18 77 29 31 37 58 60 79 66 85 70 80 43 49 21 24 95 26 41 36 14 88 54
 39  1  0 51 68 53 44 33  2 20 78 98 93 89 59 52  4  9 32 56 82 91 92 64
 57 22 27]
Cost of the best solution: 72975.19790192926
Average time per generation: 0.12704849243164062


In [ ]:
# Run 3: population_size=450, max_gen=6, elite_count=22, mutation_prob=0.045
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=450, max_gen=6, elite_count=22, mutation_prob=0.045):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/rat99.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=450, max_gen=6, elite_count=22, mutation_prob=0.045)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 7252.731672293488
Generation 2: Best Cost = 7252.731672293488
Generation 3: Best Cost = 7252.731672293488
Generation 4: Best Cost = 7252.731672293488
Generation 5: Best Cost = 7252.731672293488
Generation 6: Best Cost = 7252.731672293488
Best solution found: [23 15 46 13 75 71 63 14 27 17 33 20 35 53 59 26 25 65 81 11 92 40 50  1
 87 10  7 84 88 68 89 96 49  4  5  9 73 91 22 19 70 36 77 93 41 16 28 72
 76 80 79 95 69 58 57 34 48 32 61 62  3 43 24 38 37 86 31 51 45  6 44 47
 64 60 29  2 42 66 83 90 30 12  0  8 52 94 56 74 54 67 97 85 39 21 18 82
 98 78 55]
Cost of the best solution: 72527.31672293489
Average time per generation: 0.6409533818562825


In [ ]:
# Run 4: population_size=350, max_gen=6, elite_count=17, mutation_prob=0.035
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=350, max_gen=6, elite_count=17, mutation_prob=0.035):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/rat99.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=350, max_gen=6, elite_count=17, mutation_prob=0.035)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 7312.6750886424825
Generation 2: Best Cost = 7312.6750886424825
Generation 3: Best Cost = 7312.6750886424825
Generation 4: Best Cost = 7312.6750886424825
Generation 5: Best Cost = 7312.6750886424825
Generation 6: Best Cost = 7312.6750886424825
Best solution found: [68 83 63 30 28 58 67 80 41 85  0 38  4  2 14  9 49 46 78 29 59 55 73 34
 71 84 93 89 21 57 33 18 92 88 48 44 27 69 52 23 66 90 61  6 13  7 82 79
 11 62 51 87 47 94 95 10 50  5  1 42 65 76 45 35 22 24  3 56 77 26 12 20
 64 40 31 53 60 75 43 15 39 98 86 81 72 91 70 36 54 74 17 25  8 32 19 16
 97 37 96]
Cost of the best solution: 73126.75088642482
Average time per generation: 0.41603406270345056


In [ ]:
# Run 5: population_size=250, max_gen=6, elite_count=12, mutation_prob=0.025
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=250, max_gen=6, elite_count=12, mutation_prob=0.025):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/rat99.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=250, max_gen=6, elite_count=12, mutation_prob=0.025)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 7110.658161416487
Generation 2: Best Cost = 7110.658161416487
Generation 3: Best Cost = 7110.658161416487
Generation 4: Best Cost = 7110.658161416487
Generation 5: Best Cost = 7110.658161416487
Generation 6: Best Cost = 7110.658161416487
Best solution found: [84 95 98 61 38  1 50 10  2 14 71 16 88 67 65 27 34 19 18  6 90 91 56 43
 53 45 60  8 12 40  9  0 93 94 42 15 20 52 77 46 24 55 36 96 89 47 41 81
 64 29 97 76 75 48  7  4 63 83 86 39 51 35 23 70 13 31 30  5 72 59 73 82
 26  3 25 28 37 69 92 44 22 32 57 78 17 66 80 62 74 49 11 21 68 58 33 85
 54 79 87]
Cost of the best solution: 71106.58161416487
Average time per generation: 0.3191634019215902


In [ ]:
# Run 6: population_size=150, max_gen=6, elite_count=30, mutation_prob=0.06
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=150, max_gen=6, elite_count=30, mutation_prob=0.06):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/rat99.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=150, max_gen=6, elite_count=30, mutation_prob=0.06)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 7448.362771663795
Generation 2: Best Cost = 7448.362771663795
Generation 3: Best Cost = 7448.362771663795
Generation 4: Best Cost = 7448.362771663795
Generation 5: Best Cost = 7448.362771663795
Generation 6: Best Cost = 7448.362771663795
Best solution found: [93 76 77 55 23 39 63 84 64 16 52 31 61 98 89 94 29 28 35 40 25 44  3 10
 53  8 17 83 88 42 34 68  7 54 92 56 47 19 45 11 95 78  9 15 75 60 96 12
  4  6 32 38  2 82 72 74 26 71 21 48 67 30 90 65 85 97 36 50  5 62 14 66
 58 80 69 20 79 27 22  0 24 49 33 46 51 59 57 86 87 43 37  1 81 91 41 73
 13 18 70]
Cost of the best solution: 74483.62771663794
Average time per generation: 0.17235398292541504


In [ ]:
# Run 7: population_size=500, max_gen=6, elite_count=15, mutation_prob=0.01
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=500, max_gen=6, elite_count=15, mutation_prob=0.01):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/rat99.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=500, max_gen=6, elite_count=15, mutation_prob=0.01)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 7111.209019643049
Generation 2: Best Cost = 7111.209019643049
Generation 3: Best Cost = 7111.209019643049
Generation 4: Best Cost = 7111.209019643049
Generation 5: Best Cost = 7111.209019643049
Generation 6: Best Cost = 7111.209019643049
Best solution found: [ 7 61 37 26 38 44 49 92 82 90 69 65 54 34  9 35 39 78 97 60 87 79 86 17
 45 75  8 29 81 83 50 76 70 41 16 74 11 53 58 51 52 63 66 77 95 96 85  2
 64 40 84 48 68  0 47 36 46 80 67 24 91 18  4 23  3 31 42 33 43 98 55 89
 73 19 28 25 32 27 14 22 20 71 59 57 12 15 30 72  5  6 21 10 13  1 88 94
 56 62 93]
Cost of the best solution: 71112.0901964305
Average time per generation: 0.8292663097381592


In [ ]:
# Run 8: population_size=300, max_gen=6, elite_count=20, mutation_prob=1
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=300, max_gen=6, elite_count=20, mutation_prob=1):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/rat99.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=300, max_gen=6, elite_count=20, mutation_prob=1)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 7198.016657768042
Generation 2: Best Cost = 7198.016657768042
Generation 3: Best Cost = 7198.016657768042
Generation 4: Best Cost = 7198.016657768042
Generation 5: Best Cost = 7198.016657768042
Generation 6: Best Cost = 7198.016657768042
Best solution found: [79 88 40 58 57 91 22 28 38 15 66 42 46 36 60 30 82 76 35 34 29 27 62 11
  9 59  0 23 45 18 95 83 74 84 17 53 54  4 31 92 47  3 19 75 65 89 14 25
 93 68 72 43  7  6  2 69 98 63 10 12 26 41 49 96 37 87 48 50 80 78 70 77
 52 61  8 44 67 55  5 90 81 85 64 94 86 32 56  1 21 13 20 73 97 24 16 33
 39 71 51]
Cost of the best solution: 71980.16657768041
Average time per generation: 2.027860482533773


In [ ]:
# Run 9: population_size=600, max_gen=6, elite_count=20, mutation_prob=0.01
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=600, max_gen=6, elite_count=20, mutation_prob=0.01):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/rat99.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=600, max_gen=6, elite_count=20, mutation_prob=0.01)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 7046.774365188649
Generation 2: Best Cost = 7046.774365188649
Generation 3: Best Cost = 7046.774365188649
Generation 4: Best Cost = 7046.774365188649
Generation 5: Best Cost = 7046.774365188649
Generation 6: Best Cost = 7046.774365188649
Best solution found: [28 52 96 97 77 89 46 82 22 51 78 53 39 80 23 79 49 18  2 27 87 62 35 65
 74 90  8 69 17 15 61 88 66 38 29 55 32 86  6 95 21 60 45 37 43 41 10 33
 57 54  1  3  5 70 94 71 50 91 93 63 13 14  4  9 12 36 56 92 47 44 75 67
 81 84 72 64 16 73 83 19 20 85 34 31 98 76 48 59 40  7 26 25  0 11 30 24
 42 68 58]
Cost of the best solution: 70467.7436518865
Average time per generation: 1.0010552406311035


In [ ]:
# Run 10: population_size=700, max_gen=6, elite_count=500, mutation_prob=0.01
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=700, max_gen=6, elite_count=500, mutation_prob=0.01):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/rat99.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=700, max_gen=6, elite_count=500, mutation_prob=0.01)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 7089.460521054444
Generation 2: Best Cost = 7089.460521054444
Generation 3: Best Cost = 7089.460521054444
Generation 4: Best Cost = 7089.460521054444
Generation 5: Best Cost = 7089.460521054444
Generation 6: Best Cost = 7089.460521054444
Best solution found: [ 9 21 14 26 48 16 34 51 84 70 28 44 77 87 31 45  8 15 41 95 65  5 17 20
 11 32 49 25 86 80 67 18 35 54 93 30 91 42 63 55 10 53 29 19 52 94 75 96
 89 76 60 39 13 24  0  4 36 58 62  7 61 98 97 90 78 74 56 43 22  6 47 33
 69 27 64 40 79 57 59 81 66 82 37 38 71 50  2 73 85 68 23  3 92 83  1 46
 88 72 12]
Cost of the best solution: 70894.60521054444
Average time per generation: 0.2879134813944499
